In [1]:
import pandas as pd

In [7]:
df = pd.read_csv("/Users/sameerhussain/Desktop/CAP490/project/cleaned_dataset/cleaned_fraud_detection_file.csv")
df

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,hour,is_night,amount_ratio,sender_balance_change,receiver_balance_change,orig_balance_zero,dest_balance_zero,type_TRANSFER
0,1,9839.64,170136.00,160296.36,0.0,0.0,0,1,1,0.057834,9839.64,0.0,0,1,0
1,1,1864.28,21249.00,19384.72,0.0,0.0,0,1,1,0.087731,1864.28,0.0,0,1,0
2,1,181.00,181.00,0.00,0.0,0.0,1,1,1,0.994505,181.00,0.0,0,1,1
3,1,11668.14,41554.00,29885.86,0.0,0.0,0,1,1,0.280788,11668.14,0.0,0,1,0
4,1,7817.71,53860.00,46042.29,0.0,0.0,0,1,1,0.145146,7817.71,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684399,742,63416.99,63416.99,0.00,0.0,0.0,1,22,0,0.999984,63416.99,0.0,0,1,1
2684400,743,1258818.82,1258818.82,0.00,0.0,0.0,1,23,0,0.999999,1258818.82,0.0,0,1,1
2684401,743,339682.13,339682.13,0.00,0.0,0.0,1,23,0,0.999997,339682.13,0.0,0,1,1
2684402,743,6311409.28,6311409.28,0.00,0.0,0.0,1,23,0,1.000000,6311409.28,0.0,0,1,1


## FULL TRAINING + SAVING CODE

In [ ]:
# ===============================
# 1️⃣ Imports
# ===============================
import pandas as pd
import numpy as np
import joblib
import json
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# ===============================
# 2️⃣ Define X and y
# ===============================
X = df.drop('isFraud', axis=1)
y = df['isFraud']

# Save feature names (IMPORTANT for deployment)
feature_names = list(X.columns)

# ===============================
# 3️⃣ Train-Test Split
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ===============================
# 4️⃣ Scaling
# ===============================
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ===============================
# 5️⃣ Random Forest Model
# ===============================
rf_model = RandomForestClassifier(
    n_estimators=100,
    class_weight={0: 1, 1: 10},
    random_state=42
)

rf_model.fit(X_train_scaled, y_train)

# ===============================
# 6️⃣ Evaluation
# ===============================
y_pred = rf_model.predict(X_test_scaled)
y_prob = rf_model.predict_proba(X_test_scaled)[:, 1]

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print("ROC-AUC Score:", roc_auc_score(y_test, y_prob))


## SAVE MODEL FOR BACKEND DEPLOYMENT

In [ ]:
# ===============================
# 7️⃣ Save Everything
# ===============================

os.makedirs("models", exist_ok=True)

# Save model
joblib.dump(rf_model, "models/random_forest_model.joblib")

# Save scaler
joblib.dump(scaler, "models/scaler.joblib")

# Save feature names
with open("models/feature_names.json", "w") as f:
    json.dump(feature_names, f)

print("✅ Model, Scaler, and Feature Names Saved Successfully!")
